# Welcome to Flow!

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

import flow

data = flow.configure_data()


In [3]:
df = flow.calc_electricity_wastewater(data=data, total=True)
df

,FIPS,State,County,electricity_wastewater_total_bbtu,wastewater_rejected_energy_total_bbtu,wastewater_energy_services_total_bbtu
0,01001,AL,Autauga County,0.026169,0.009159,0.017010
1,01003,AL,Baldwin County,0.115393,0.040388,0.075006
2,01005,AL,Barbour County,0.015174,0.005311,0.009863
3,01007,AL,Bibb County,0.005209,0.001823,0.003386
4,01009,AL,Blount County,0.009898,0.003464,0.006434
...,...,...,...,...,...,...
3219,72151,PR,Yabucoa Municipio,0.007026,0.002459,0.004567
3220,72153,PR,Yauco Municipio,0.011427,0.003999,0.007427
3221,78010,VI,St. Croix County,0.032860,0.011501,0.021359
3222,78020,VI,St. John County,0.000000,0.000000,0.000000


In [8]:
df = flow.calc_electricity_wastewater(data=data)
df

,FIPS,State,County,electricity_wastewater_advanced_bbtu,wastewater_advanced_rejected_energy_bbtu,wastewater_advanced_energy_services_bbtu,electricity_wastewater_secondary_bbtu,wastewater_secondary_rejected_energy_bbtu,wastewater_secondary_energy_services_bbtu,electricity_wastewater_primary_bbtu,wastewater_primary_rejected_energy_bbtu,wastewater_primary_energy_services_bbtu,electricity_wastewater_total_bbtu,wastewater_rejected_energy_total_bbtu,wastewater_energy_services_total_bbtu
0,01001,AL,Autauga County,7532.00,2636.200,4895.800,137.28,48.048,89.232,0.0,0.0,0.0,7669.28,2684.248,4985.032
1,01003,AL,Baldwin County,18830.00,6590.500,12239.500,14988.48,5245.968,9742.512,0.0,0.0,0.0,33818.48,11836.468,21982.012
2,01005,AL,Barbour County,0.00,0.000,0.000,4447.04,1556.464,2890.576,0.0,0.0,0.0,4447.04,1556.464,2890.576
3,01007,AL,Bibb County,0.00,0.000,0.000,1526.72,534.352,992.368,0.0,0.0,0.0,1526.72,534.352,992.368
4,01009,AL,Blount County,2399.48,839.818,1559.662,501.28,175.448,325.832,0.0,0.0,0.0,2900.76,1015.266,1885.494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,72151,PR,Yabucoa Municipio,0.00,0.000,0.000,2059.20,720.720,1338.480,0.0,0.0,0.0,2059.20,720.720,1338.480
3220,72153,PR,Yauco Municipio,0.00,0.000,0.000,3348.80,1172.080,2176.720,0.0,0.0,0.0,3348.80,1172.080,2176.720
3221,78010,VI,St. Croix County,0.00,0.000,0.000,9630.40,3370.640,6259.760,0.0,0.0,0.0,9630.40,3370.640,6259.760
3222,78020,VI,St. John County,0.00,0.000,0.000,0.00,0.000,0.000,0.0,0.0,0.0,0.00,0.000,0.000


In [ ]:
df = flow.calc_sectoral_use_energy_discharge(data=data, total=True)
df

In [6]:
for col in data.columns:
    print(col)


FIPS
State
County
population
fresh_groundwater_pws_mgd
fresh_surface_water_pws_mgd
saline_groundwater_pws_mgd
saline_surface_water_pws_mgd
pws_residential_mgd
total_pws_mgd
fresh_groundwater_residential_mgd
fresh_surface_water_residential_mgd
fresh_groundwater_thermoelectric_mgd
saline_groundwater_thermoelectric_mgd
fresh_surface_water_thermoelectric_mgd
saline_surface_water_thermoelectric_mgd
wastewater_thermoelectric_mgd
pws_thermoelectric_mgd
thermoelectric_consumption_mgd
fresh_groundwater_industrial_mgd
fresh_surface_water_industrial_mgd
saline_groundwater_industrial_mgd
saline_surface_water_industrial_mgd
fresh_groundwater_mining_mgd
fresh_surface_water_mining_mgd
saline_groundwater_mining_mgd
saline_surface_water_mining_mgd
fresh_groundwater_crop_irrigation_mgd
fresh_surface_water_crop_irrigation_mgd
wastewater_crop_irrigation_mgd
crop_irrigation_freshwater_consumption_mgd
fresh_groundwater_golf_irrigation_mgd
fresh_surface_water_golf_irrigation_mgd
wastewater_golf_irrigation_mg

In [ ]:
data.to_csv(r'C:\Users\mong275\Local Files\Sankey_Notebook_Development\SankeyFiles\run4.csv')

To override the data inputs, flow.configure_data() can be run.
If run with no arguments provided, this function aggregates all baseline data to calculate flow values for the united states in 2015. Parameters are set, however, such that this function can be used to swap out datafiles. 

For example, if alternative US county level data was available for interbasin transfer electricity use, that could be used instead of the default data.

In [ ]:
df = flow.configure_data()
df

In [ ]:
is_NaN = df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df[row_has_NaN]
rows_with_NaN

In [ ]:
df2.to_csv(r'C:\Users\mong275\Local Files\Sankey_Notebook_Development\SankeyFiles\run2.csv')

<h1> Public Water Supply </h1>

## Water in Public Water Supply

### Public Water Supply Withdrawals

The USGS 2015 dataset provides public water supply withdrawals directly under data codes 'PS-WGWFr', 'PS-WSWFr','PS-WGWSa', 'PS-WSWSa' for each county.

In [7]:
df = flow.prep_water_use_2015(variables=['State','County','PS-WGWFr', 'PS-WSWFr','PS-WGWSa', 'PS-WSWSa'])
df.head()               

KeyError: "['PS-WGWFr', 'PS-WSWFr', 'PS-WGWSa', 'PS-WSWSa'] not in index"

### Public Water Supply Deliveries to Sectors, Exports, and Imports

The USGS 2015 dataset provide public water supply data in three categories: (1) total public water supply withdrawals, (2) public water supply deliveries to domestic end users, and (3) public water supply deliveries to thermoelectric cooling.

The 1995 USGS dataset provides public water supply deliveries to domestic, industrial, commercial, thermoelectric cooling, and losses/exports separately. To estimate the deliveries to commercial and industrial customers in 2015, the ratio of each to the sum of deliveries to residential and thermoelectric cooling is calculated from the 1995 data. These ratios are applied to the 2015 residential and thermoelectric cooling public water supply delivery values. 

Additionally, public water suppy exports and imports are calculated. When the sum of water deliveries is greater than the water withdrawn for public water supply, the difference is considered to be imported. When water withdrawn is greater, excess water after deliveries is considered to be exported.

Caveats and assumptions:
* This methodology assumes that if no public water supply was delivered to residential or thermoelectric cooling in 1995, then deliveries to industrial and commercial sectors would likewise be zero.

Potential mods
 for import pws counties, take 15% of withdrawals and imports as losses and add it on to the top of imports
 for export pws counties, take 15% of withdrawals as losses, take it out of exports, any remaining put as imports

In [ ]:
df = flow.calc_pws_discharge()
df.head()

## Energy in Public Water Supply

Energy in public water supply includes
groundwater pumping, surface water pumping, desalination (when applicable), surface water treatment, groundwater treatment, distribution, and interbasin transfers (when appropriate).

An assumption is made that water that goes through desalination additionally requires surface water or groundwater treatment

The same groundwater and surface water pumping intensities are used from agriculture sector.

Note that interbasin transfer data that is provided to this function is already in bbtu, not in water flows. 
For non-ibt data, the option is provided for the user to use region-level energy intensity values (as in groundwater and surface water pumping). When values are not available, assumptions for each in kWh/MG are set as default values that can be overriden.

intensity values are used from tidwell and LLNL

In [ ]:
df = flow.calc_electricity_public_water_supply(total=True)
df.head()

# Wastewater Treatment
Wastewater treatment facility data is provided by EPA for the year 2012. These values are assumed for 2015. Values are provided on a treatment plant level with data on the following variables: 
Flow:
    * Total water flows (MGD) treated by the facility
    * Total infiltration flows treated by the facility (leaks into pipes and conduits)
Treatment:
    * Treatment type of each treatment plant (primary, secondary, advanced)
Discharge:
    * discharge location of each plant, broken down by percent of flow to each discharge location (e.g., ocean discharge, suface discharge, irrigation reuse)
Location:
    * county information is included for each power plant
    
This data allows us to determine total water flows processed and discharged by wastewater facilities at a county level by treatment type. The treatment type allows us to calculate the electricity use by county for wastewater treatment,

## Water in Wastewater Treatment

### Water Inflows to Wastewater Treatment Facilities
Total flows processed by wastewater treatment facilities are provided in the EPA dataset directly. However, this data will have discrepencies from the water data delivered to various sectors. To estimate water flows from residential, commercial, and industrial sectors to wastewater treatment facilities, an assumption is made that all water delivered from public water supply to these sectors is sent to wastewater treatment facilities. This assumption is also used to fill wastewater treatment flow values for regions that are unavailable in the EPA dataset. For example, south carolina does not have wastewater treatment facility values in the EPA dataset. It is assumed, therefore, that total wastewater treatment flows in south carolina and equal to the total public water supply flows for each county and is attributed to municipal discharge rather than infiltration. It is assumed that 80% of treatment flows are advanced treatment and 20% are secondary treatment. It is also assumed that all discharge is sent to surface. This follows average values for the state of North Carolina which were used as a proxy in this instance.

### Quantity of Wastewater Processed by Treatment Type
Wastewater treatment plant treatment type is provided in the EPA data directly. These values are binned for simplification into no treatment, primary, secondary, and advanced treatment. When no treatment type was provided for a facility, it was assumed that they were secondary treatment facilities.

To see the amount of water (MGD) processed within each county by treatment type, run the cell below.

In [ ]:
df = flow.prep_wastewater_data()
df.head()

### Wastewater Treatment Facility Discharge

Wastewater treatment plant discharge location is provided in the EPA data directly. These values are binned for simplification into the following categories:

* 'outfall to surface waters': 'ww_sd',
* 'ocean discharge': 'ww_od',
* 'deep well': 'ww_gd',
* "reuse: industrial": 'ww_in',
* 'evaporation': 'ww_evap',
* 'spray irrigation': 'ww_ir',
* 'overland flow no discharge': 'ww_ww',
* 'overland flow with discharge': 'ww_sd',
* 'discharge to another facility': 'ww_ww',
* 'combined sewer overflow (cso) discharge': 'ww_sd',
* 'other': 'ww_sd',
* 'discharge to groundwater': 'ww_gd',
* 'no discharge, unknown': 'ww_ww',
* 'reuse: irrigation': 'ww_ir',
* 'reuse: other non-potable': 'ww_sd',
* 'reuse: indirect potable': 'ww_sd',
* 'reuse: potable': 'ww_ps',
* 'reuse: groundwater recharge': 'ww_gd'

In [ ]:
df = flow.prep_wastewater_data()
df = df[['FIPS','ww_evap','ww_gd','ww_in','ww_ir','ww_od','ww_ps','ww_sd','ww_ww']]
df.head()

## Energy in Wastewater Treatment

Wastewater treatment facilities have various levels of treatment from primary to advanced. Each treatment level has an increasing level of energy intensity to process the water. Intensity value estimates for wastwater treatment were used from Pabi et al. [17].

# Agriculture

## Water in Agriculture

## Energy in Agriculture

Energy use in agriculture consists of groundwater pumping, surface water pumping, and interbasin transers. USDA's Farm and Ranch Irrigation Survey (FRIS) provides state-by-state data on irrigation groundwater depth and average irrigation pressurization levels for irrigation within a state, enabling the calculation of pump electricity consumption for both groundwater and surface water pumping [13]. The 2013 survey is the closest year available to 2015 values. It is assumed that values do not vary significantly between the two years.

The methodology for calculating groundwater and surface water pumping energy is described in Tidwell et al [14]. However, where that publication uses well depth to water to calculate total differential height, the total well depth is used here as a way to offset some of the losses due to friction that would occur in the piping, as described in Lawrence Berkeley National Laboratory (LBNL) Home Energy Saver & Score: Engineering Documentation [15].

Pump efficiency is assumed to be the average (46.5%) of the range (34-59%) listed in [14]. State-level intensity rates are calculated here and applied to the county level water in agriculture values. The general methodology implemented here to determine total energy in agriculture is adapted from [5], not including the additions previously mentioned.

### Groundwater & Surface Water Pumping

### Interbasin Transfers

# Electricity Generation

## Energy in Electricity Generation

US Energy Information Administration (EIA) 923 data for 2015 is used to determine electricity generation by generator type within each interconnect [12]. The datafile contains information on generation at the plant-level. EIA additionally provides data on which North American Electric Reliability Corporation (NERC) balancing region each plant belongs to. By mapping each NERC region to its associated interconnect, generation within each interconnect can be determined directly.

From EIA, the AER Fuel Type Code is used to determine generator type. EIA provides generator type information at a high level of detail. For this reason different generator types are grouped for simplicity using a generator key (GenType923Key.xlsx). For example, different types of natural gas generators are all grouped into "Natural Gas" as a type.

Energy production is provided as "Net Generation (Megawatthours)" in the EIA dataset, "Total Fuel Consumption (MMBTU)" is the MMBTU equivalent of fuel going to each plant for electricity generation. The difference between these two values indicates the amount of energy generation that does not go to energy losses.

## will be filled in with calculate function

## Water in Electricity Generation

# End Use Sectors

## Residential Sector

### Water in Residential Sector

### Energy in Residential Sector

#### Energy (electricity) Demand in Residential Sector

##### weighted by population. <b> consider splitting out by usage if data is available

In [ ]:
df = flow.prep_electricity_demand_data()
df

#### Energy (fuel) Demand in Residential Sector

## Commercial Sector

### Water in Commercial Sector

### Energy in Commercial Sector

## Industrial Sector

### Water in Industrial Sector

### Energy in Industrial Sector

## Transportation Sector

### Energy in Energy Sector

# Energy Production

## Biomass

### Energy in Biomass Production (Ethanol)

To find the ethanol production per county, this calculation takes 2015 eia data on ethanol plant capacity which has added locational data by each plant and combines with state level biomass (ethanol) production from prep_state_fuel_production_data() to split out state total by county. Returns a dataframe of ethanol production (bbtu) by county FIPS for each county in the US for 2015.

ethanol plant production filler for wyoming (in state total but not location data) ,Torrington, WY100%, https://neo.ne.gov/programs/stats/122/2015/122_201512.htm

#### <b> <ul> <font color='red'> Consider looking for estimates for electricity use in biomass production to split out from industrial

### Water in Biomass Production (Ethanol)

#### Water used in irrigation of corn used for ethanol

Data on water used in the irrigation of corn ultimately used for ethanol production does not exist explicitly at any level of granularity and must be calculated out from larger state-level corn growth irrigation intensity estimates.

The percent of corn grown for ethanol in the US was used from US DOE Alternative Fuels Data Center [21]. Total corn production for ethanol (5.22 billion bushels) was divided by total corn producton (13.6 billion bushels) for 2015 and calculated to be 38.406%. Data on a state or county-level does not exist, therefore, this percentage is applied to all states with corn production. A state-level fraction of corn used for ethanol production could be calculated from the amount of ethanol produced in a state and applying an assumption of corn required per unit of ethanol, however, not all corn used in ethanol is produced in the state that uses it which lead to states such as CA with an ethanol fraction of corn growth of nearly 900% of their total corn production. Given that we are unable to determine which states the imported corn for ethanol would be coming from, the decision was made to apply the US total fraction to all states.

Total water used in the production of corn for ethanol is therefore the assumed 38.406% multiplied by the product of the total amount of irrigated acres of corn (AC_Irr_Corn) and the calculated irrigation intensity for that state (total acres applied to all crops / total acres irrigated, all crops) provided from USDA Census of Agriculture Irrigation and Water Management Survey [22], Table 35 and Table 4, respectively. The irrigation intensity for corn is calculated from the average irrigation intensity for the state.

To split the total MGY applied between fresh surface and groundwater, USDA FRIS Census of Agriculture data on the fraction of surface water (surface water + off farm water) from total water supply (including groundwater) in corn production is used from Table 37 of [22]. Off-farm sources are assumed to be surface water.

Some states that grow corn do not provide data on the water source breakdown (surface vs. ground) for their irrigation. The surface to groundwater ratio for all irrigation in a state is substituted for these states. These values are calculated from the USGS 2015 water dataset.

In [ ]:
df = flow.prep_county_water_corn_biomass_data()

#### Water used in the production of ethanol (fermentation)

#### <b> <ul> <font color='red'> Consider looking for estimates for water used in the fermentation process of ethanol production, not just corn growth.

## Petroleum

### Energy in Petroleum Production (Conventional and Unconventional Oil Drilling)

 crude oil production by county fillers
Idaho: https://ogcc.idaho.gov/wp-content/uploads/sites/3/2017/06/2016-10-6-state-of-idaho-releases-oil-gas-production-data.pdf
100% of Idaho petroleum and natural gas is from Payette County

Alaska,
https://www.commerce.alaska.gov/web/aogcc/Data.aspx
almost all oil is produced along the north slope. Calculated from oil production download. The rest is on the kenai peninsula (cook inlet basin)

Maryland
https://www.eia.gov/state/analysis.php?sid=MD#34
assumed 50/50 split between Garret and Allegany Counties

Nevada
https://gisweb.unr.edu/OilGas/
Assumed to be in nye county with producing oil wells (only 4 btu)

Oregon
https://www.oregongeology.org/mlrr/oilgas-report.htm
Only one county (columbia county)

### Water in Petroleum Production (Conventional and Unconventional Oil Drilling)

## Natural Gas

### Energy in Natural Gas Production (Unconventional Drilling)

### Water in Natural Gas Production (Unconventional Drilling)

## Coal

### Energy in Coal Production (Coal Mining)

### Water in Coal Production (Coal Mining)

# Intensities & Percentages

#### Conveyance Losses as a Percent of Water Withdrawn for Irrigation
Conveyance loss fraction for irrigation is calculated from the USGS 1995 water use dataset. It is calculated as the total conveyance losses (MGD) divided by the total water withdrawn (MGD) for irrigation. Looking at the fractions at the county level shows that a small subset of counties have unreliable values where conveyance losses are far greater than the amount withdrawn for irrigation. As a result, the ability to cap the loss fraction at a specified amount it provided and replace it with the average of the specified region. The default loss cap fraction is .90.

In [ ]:
df = flow.prep_water_use_1995()
df = flow.calc_conveyance_loss_frac(df)
df.head()

##### Change in loss cap

In [ ]:
df = flow.prep_water_use_1995()
df = flow.calc_conveyance_loss_frac(df, loss_cap_amt=.20)
df.head()

In [ ]:
help(flow.plot_bar)

In [ ]:
flow.plot_bar(df, "State","IR_CLoss_Frac", "State", y_axis_title = "Irrigation Conveyance Loss Fraction", x_axis_title="State")

#### Water Use Intensity for Hydroelectric Generation

In [ ]:
df = flow.calc_hydroelectric_water_intensity(intensity_cap=True, intensity_cap_amt=165, region_avg=False)

df.head()

In [ ]:
flow.plot_bar(df, "State","HY_IF", "State")